In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, TensorDataset

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch")

In [2]:
## uncomment for colab
## upload datasetGenerator.py for preprocessing dataset

# from google.colab import drive
# drive.mount('/content/drive')
# ! unzip -q "/content/drive/MyDrive/Colab Notebooks/BVP.zip"
# ! python /content/datasetGenerator.py

In [3]:
fraction_for_test = 0.2
num_class = 3
ALL_MOTION = [i for i in range(1, num_class+3)]
N_MOTION = len(ALL_MOTION) # Number of output classes
T_MAX = 38 # Number of timestamps
n_gru_hidden_units = 128
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Model

In [4]:
class CNNModule(nn.Module):
    def __init__(self):
        super(CNNModule, self).__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=8, kernel_size=2, padding='same'),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(8 * 10 * 10, 64),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(64, 32),
            nn.ReLU()
        )

    def forward(self, x):
        return self.cnn(x)

class ConvGRUModel(nn.Module):
    def __init__(self, hidden_size, num_classes, num_timestamps):
        super(ConvGRUModel, self).__init__()
        
        # CNN module for each input timestamp
        self.cnn_modules = nn.ModuleList([
            CNNModule() for _ in range(num_timestamps)
        ])
        
        # GRU layers
        self.gru = nn.GRU(32, hidden_size, num_layers=num_timestamps, batch_first=True, dropout=0.25)

        # Fully connected layer at the output of last GRU
        self.fc_out = nn.Linear(hidden_size, num_classes)

        # Relu activation for fully connected
        self.relu = nn.ReLU()
        # Softmax activation for classification
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # Apply CNN module sequentially for each timestamp
        x = np.swapaxes(x, 0, 1)
        x = [module(xi) for module, xi in zip(self.cnn_modules, x)]
        x = torch.stack(x, dim=1)  # Stack along the time dimension
        
        # GRU layer
        x, _ = self.gru(x)

        # Apply ReLU activation after the GRU layer
        x = self.relu(x)

        # Fully connected layer at the output of last GRU
        x = self.fc_out(x[:, -1, :])
        
        # Softmax for classification
        x = self.softmax(x)

        return x

## Load dataset

In [5]:
# Load datasets
num_clients = 5
batch_size = 8
client_datasets = {}
client_loaders = {}

for i in range(1, num_clients + 1):
    # Load client data
    client_data = torch.load(f'./data/data{i}.pt')
    data = torch.from_numpy(client_data['data']).float()
    label = torch.from_numpy(client_data['label']).long()

    # Split data into training and testing sets
    data_train, data_test, label_train, label_test = train_test_split(
        data, label, test_size=fraction_for_test, random_state=42
    )

    train_dataset = TensorDataset(data_train, label_train)
    test_dataset = TensorDataset(data_test, label_test)
    client_datasets[f'client{i}'] = {'train': train_dataset, 'test':test_dataset}

    # Set up data loaders for each client's
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    client_loaders[f'client{i}'] = {'train': train_loader, 'test':test_loader}

## FedAvg

In [6]:
class FedAvgAlgorithm:
    def __init__(self, model, n_gru_hidden_units, num_class, T_MAX, train_loader):
        self.model = model
        self.global_model = model(n_gru_hidden_units, num_class, T_MAX)
        self.train_loader = train_loader
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    def train(self, model, device, train_loader, optimizer, criterion):
        model.train()
        train_loss = 0
        correct = 0
        total = 0

        for data, target in train_loader:
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            train_loss += loss.item()
            loss.backward()
            optimizer.step()
        
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

        train_loss /= len(train_loader)
        train_accuracy = 100 * correct / total
        return train_loss, train_accuracy

    def test(self, model, device, test_loader, criterion):
        model.eval()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target)
                test_loss += loss.item()
                _, predicted = output.max(1)
                correct += predicted.eq(target).sum().item()

        test_loss /= len(test_loader)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        return test_loss, test_accuracy

    def run(self, num_rounds, num_epochs):
        result = []
        round_accuracy_all = []
        for round in range(num_rounds):
            print(f"\n---------- Round {round + 1}/{num_rounds} ----------")

            local_model_updates = []
            client_results = {'loss':[], 'accuracy':[]}

            for client_id in range(1, len(client_loaders)+1):
                print(f"\nTraining on Client {client_id}")

                # Create a local copy
                local_model = self.model(n_gru_hidden_units, num_class, T_MAX).to(self.device)
                local_model.load_state_dict(self.global_model.state_dict())
                criterion = nn.CrossEntropyLoss()
                optimizer = optim.Adam(local_model.parameters(), lr=0.001)

                # Local training
                loss, accuracy = [], []
                for epoch in range(num_epochs):

                    train_loss, train_accuracy = self.train(local_model, self.device, self.train_loader[f'client{client_id}']['train'], optimizer , criterion)
                    val_loss, val_accuracy = self.test(local_model, self.device, self.train_loader[f'client{client_id}']['test'], criterion)

                    loss.append((train_loss, val_loss))
                    accuracy.append((train_accuracy, val_accuracy))
                    print(f'        Epoch: {epoch+1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%,', 
                          f' Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')
                    

                client_results['loss'].append(loss)
                client_results['accuracy'].append(accuracy)
                local_model_updates.append(local_model.state_dict())
                
            # Weighted average
            averaged_state_dict = {}
            client_size = [len(v['train'].dataset) for k, v in self.train_loader.items()]
            for key in self.global_model.state_dict():
                averaged_state_dict[key] = sum(client_size[ind]*update[key] for ind, update in enumerate(local_model_updates)) / np.sum(client_size)
            
            # update global model
            self.global_model.load_state_dict(averaged_state_dict)

            # calculate round test accuracy with weighted mean
            round_acc = []
            client_size = [len(v['test'].dataset) for k, v in self.train_loader.items()]
            for cl, data_ in self.train_loader.items():
                _, val_accuracy = self.test(self.global_model, self.device, data_['test'], criterion)
                round_acc.append(val_accuracy)
            round_accuracy = np.average(round_acc, weights=client_size)
            print(f'\n The round accuracy is: {round_accuracy}')
            round_accuracy_all.append(round_accuracy)
            result.append(client_results)
            
        return round_accuracy_all, result

In [7]:
fed_avg_algorithm = FedAvgAlgorithm(model=ConvGRUModel,
                                    n_gru_hidden_units=n_gru_hidden_units, 
                                    num_class=N_MOTION, 
                                    T_MAX=T_MAX, 
                                    train_loader=client_loaders
                                    )

fed_avg_result = fed_avg_algorithm.run(num_rounds=5, num_epochs=3)


---------- Round 1/5 ----------

Training on Client 1
        Epoch: 1, Train Loss: 1.5013, Train Accuracy: 32.43%,  Val Loss: 1.4904, Val Accuracy: 30.63%
        Epoch: 2, Train Loss: 1.4920, Train Accuracy: 31.40%,  Val Loss: 1.4903, Val Accuracy: 35.86%
        Epoch: 3, Train Loss: 1.4915, Train Accuracy: 32.88%,  Val Loss: 1.4984, Val Accuracy: 30.63%

Training on Client 2
        Epoch: 1, Train Loss: 1.4971, Train Accuracy: 34.28%,  Val Loss: 1.4831, Val Accuracy: 38.95%
        Epoch: 2, Train Loss: 1.4874, Train Accuracy: 36.97%,  Val Loss: 1.5023, Val Accuracy: 30.32%
        Epoch: 3, Train Loss: 1.4883, Train Accuracy: 34.12%,  Val Loss: 1.4810, Val Accuracy: 38.95%

Training on Client 3
        Epoch: 1, Train Loss: 1.4992, Train Accuracy: 33.42%,  Val Loss: 1.4968, Val Accuracy: 31.20%
        Epoch: 2, Train Loss: 1.4945, Train Accuracy: 31.69%,  Val Loss: 1.4920, Val Accuracy: 31.20%
        Epoch: 3, Train Loss: 1.4914, Train Accuracy: 32.49%,  Val Loss: 1.4989, Val 

AttributeError: 'FedAvgAlgorithm' object has no attribute 'data_'